# Skin effect
In the lectures (nugget 13) you looked at AC current in a wire and particularly the skin effect. In the process you saw a full-on (non-examinable) solution. In this worksheet you calculate an approximation for when the skin depth is much less than the radius. This worksheet plots the full on solution allowing you to inspect how well this agrees with the sketches in your approximation.

**You need to run all the cells in this section for anything to work.**

In [1]:
%matplotlib inline
from matplotlib import pyplot
import numpy as np
import ipywidgets

from scipy.special import jv as bessel # renamed Modified Bessel function of the first kind of real order. for clarity

In [7]:
def skinDepth(conductivity = 6e7, frequency = 0):
    r'''
    Calculates the skin depth for a given frequency. sets to infinity if frequency is 0
    '''
    mu0 = 4 * np.pi * 1e-7
    omega = 2 * np.pi * frequency
    
    if frequency == 0:
        delta = np.Inf
    else:
        delta = np.sqrt(2/(mu0 * conductivity * omega))
    return delta

def approxJc(x, t, frequency = 0, conductivity  = 6e7):
    r''' 
    Returns the approximate Jc as calculated in the worksheet
    '''
    delta = skinDepth(conductivity = conductivity, frequency = frequency)
    phasefactor = np.exp(- 1j * 2 * np.pi * frequency * t)
    
    jc =  np.exp(-x / delta) * np.exp( 1j * x / delta)
    Jc = np.real(jc * phasefactor)
    return Jc

def JcFullon(x, t, frequency = 0, conductivity  = 6e7):
    r'''
    Plots the approximation of Jc from the worksheet
    '''
    #x = np.arange(0,1,0.001) # distance from centre of the wire forced to be 1 (as normalised to a radius of 1)
    a = 1
    r = a - x # In the approximation we work with x but for the full solution we use r
    
    delta = skinDepth(conductivity = conductivity, frequency = frequency)
    phasefactor = np.exp(- 1j * 2 * np.pi * frequency * t)
    
    j0 = bessel(0, (1 + 1j) * 1 / delta) # here j0 = jc(x = 0) as in the worksheet)
    Jc_norm = bessel(0, (1 + 1j) * r / delta) * phasefactor / j0 
    Jc_norm = np.real(Jc_norm)
    return Jc_norm

def plotJcFullon(t, frequency = 0, conductivity  = 6e7):
    r'''
    Plots the approximation of Jc from the worksheet
    '''
    x = np.arange(0,1,0.0001) # distance from centre of the wire forced to be 1 (as normalised to a radius of 1)
    
    
    fig = pyplot.plot(x, approxJc(x,t, frequency = frequency, conductivity = conductivity))
    fig = pyplot.plot(x, JcFullon(x,t, frequency = frequency, conductivity = conductivity))
    # formatting the plot
    pyplot.ylim(-1.1,1.1)
    pyplot.xlim(0,1)
    pyplot.xlabel('$x/a$')
    pyplot.ylabel('$J_C(r)/J_0$')  
    pyplot.legend(['Approximate solution', 'Full on solution'])
    

def plotJcApprox(t, frequency = 0, conductivity  = 6e7):
    r'''
    Plots the approximation of Jc from the worksheet
    '''
    x = np.arange(0,1,0.0001) # distance from centre of the wire forced to be 1 (as normalised to a radius of 1)
    
    
    fig = pyplot.plot(x, approxJc(x,t, frequency = frequency, conductivity = conductivity))
    # formatting the plot
    pyplot.ylim(-1.1,1.1)
    pyplot.xlim(0,0.1)
    pyplot.xlabel('$x/a$')
    pyplot.ylabel('$J_C(r)/J_0$')  
    

# Approximation

In the worksheet you calculated an approximation for the current density as a function of depth (x). This is plotted in the below function.

In [3]:
ipywidgets.interact(
    lambda t, frequency, conductivity: plotJcApprox(t = t, frequency = frequency, conductivity = conductivity),
    t=ipywidgets.FloatSlider(min=0, max=400, step=0.1, value=0), #  in seconds 
    frequency= ipywidgets.FloatSlider(min=0, max=49, step=0.1, value=0.0), # in Hz
    conductivity = ipywidgets.FloatSlider(min=1e7, max=1e8, step=1e6, value=6e7), # in Hz
);

interactive(children=(FloatSlider(value=0.0, description='t', max=400.0), FloatSlider(value=0.0, description='…

# Comparison to full solution
The full on solution uses complex Bessel functions. Working with these is beyond the scope of the course but it is possible to simulate. Below the approximation from the worksheet is overlaid on the full on solution to give you an idea when this approximation breaks down. Note that the scale on the x axis is changed compared to the previous example.

In [4]:
ipywidgets.interact(
    lambda t, frequency, conductivity: plotJcFullon(t = t, frequency = frequency, conductivity = conductivity),
    t=ipywidgets.FloatSlider(min=0, max=50, step=0.1, value=0), #  in seconds 
    frequency= ipywidgets.FloatSlider(min=0, max=49, step=0.1, value=0.0), # in Hz
    conductivity = ipywidgets.FloatSlider(min=1e5, max=1e8, step=1e6, value=6e7), # in Hz
);

interactive(children=(FloatSlider(value=0.0, description='t', max=50.0), FloatSlider(value=0.0, description='f…

# Kymograph
It can be illustrative to show this function as a kymograph with time on the x axis and distance from the surface on the y axis 

In [5]:
def plotKymograph(frequency = 1, conductivity  = 6e7):
    depth = np.arange(0,0.1,0.01)
    time = np.arange(0, 10, 0.01)

    time, depth = np.meshgrid(time, depth)
    Jc = JcFullon(depth, time, frequency = frequency, conductivity  = conductivity)
    fig, axes = pyplot.subplots(figsize = [20,10])
    cs = axes.contourf(time, depth, Jc, cmap = 'Spectral')
    cbar = fig.colorbar(cs)
    axes.set_xlabel('time/s')
    axes.set_ylabel('depth in wire/a')

In [6]:
ipywidgets.interact(
    lambda frequency, conductivity: plotKymograph(frequency = frequency, conductivity = conductivity),
    frequency= ipywidgets.FloatSlider(min=0, max=10, step=0.1, value=0.0), # in Hz
    conductivity = ipywidgets.FloatSlider(min=1e7, max=1e8, step=1e6, value=6e7), # in Hz
);

interactive(children=(FloatSlider(value=0.0, description='frequency', max=10.0), FloatSlider(value=60000000.0,…